<a href="https://colab.research.google.com/github/fritzmartin003/RAG-System-Projekt/blob/main/Test_Ichwei%C3%9Fnichtweiter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Notwendige Bibliotheken installieren
!pip install faiss-cpu transformers sentence-transformers pymupdf numpy scipy
!pip uninstall -y tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.9/275.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 69.4 MB/s eta 0:00:00
Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0


In [2]:
# Imports
import fitz  # PyMuPDF\n",
import numpy as np
import faiss
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer

In [3]:
from google.colab import userdata
HuggingFaceAPIKey = "HF2"

In [6]:
# PDF-Text extrahieren\n",
def extract_text_from_pdf(pdf_path):
      text = " "
      with fitz.open(pdf_path) as doc:
        for page in doc:
          text += page.get_text("text") + " "
        return text

pdf_path = "SakowskiBuch.pdf"
pdf_text = extract_text_from_pdf(pdf_path)

In [7]:
 # Text in Chunks teilen\n",
def split_text(text, chunk_size=300, overlap=90):
    chunks = []
    start = 0
    while start < len(text):
      end = start + chunk_size
      chunks.append(text[start:end])
      start += chunk_size - overlap
    return chunks

chunks = split_text(pdf_text)
print(f" PDF in {len(chunks)} Chunks unterteilt!")

 PDF in 2969 Chunks unterteilt!


In [18]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import sklearn.preprocessing
import torch
import time


# Embedding Modell laden\n",
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Optimale Batch-Größe finden (Experimentieren!)\n",
optimal_batch_size = 3000  # Starte mit einem hohen Wert und reduziere, wenn du Speicherprobleme hast\n",

def create_faiss_index(chunks):
      start_time = time.time()  # Zeitmessung\n",

# Embeddings erstellen (mit optimierter Batch-Verarbeitung und Tensor-Konvertierung)\n",
      chunk_embeddings_tensor = embedding_model.encode(chunks, batch_size=optimal_batch_size, convert_to_tensor=True)
      chunk_embeddings = chunk_embeddings_tensor.cpu().numpy()

# Normalisieren der Embeddings (wichtig für die meisten Distanzmetriken)\n",
      chunk_embeddings = sklearn.preprocessing.normalize(chunk_embeddings)

      dimension = chunk_embeddings.shape[1]

# FAISS Index erstellen (IndexIVFFlat mit Training)\n",
      nlist = int(np.sqrt(len(chunk_embeddings)))  # Anzahl der Partitionen (Faustregel)\n",
      quantizer = faiss.IndexFlatL2(dimension)  # Quantisierer für die Clusterzentren\n",
      index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_L2)

# Training des Index (NOTWENDIG für IndexIVFFlat!)\n",
      index.train(chunk_embeddings)

# Hinzufügen der Embeddings zum Index (NACH dem Training)\n",
      index.add(chunk_embeddings)

      end_time = time.time()
      print(f"FAISS Vektordatenbank erstellt! (Zeit: {end_time - start_time:.2f} Sekunden)")
      return index

# Beispielverwendung (ersetze durch deine tatsächlichen chunks)\n",
      chunks = [
          "Dies ist ein kurzer Textabschnitt.",
          "Ein weiterer, etwas längerer Textabschnitt mit mehr Informationen.",
        # ... deine chunks hier ...\n",
        ],

      index = create_faiss_index(chunks)

In [16]:
from google.colab import userdata
HuggingFaceAPIKey = "HF2"
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def get_embedding(text):
        return embedding_model.encode(text, convert_to_numpy=True)

        chunk_embeddings = np.array(embedding_model.encode(chunks, batch_size=512, convert_to_numpy=True))


In [17]:
 # FAISS Vektordatenbank aufsetzen\n",
dimension = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(chunk_embeddings)
print(" FAISS Vektordatenbank erstellt!")

NameError: name 'chunk_embeddings' is not defined

In [ ]:
# Ähnlichkeitssuche in FAISS\n",
def get_relevant_chunks(query, top_k=3):
    query_embedding = get_embedding(query).reshape(1, -1)
    distances, indices = index.search(query_embedding, top_k)
    return [chunks[i] for i in indices[0]]

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import os
import torch

!huggingface-cli login

os.environ["HUGGINGFACE_API_KEY"] = "HF2"
model_name = "mistralai/Mistral-7B-v0.1"  # Alternativ: \"meta-llama/Llama-2-7b-chat-hf\"\n",

# Tokenizer & Modell mit explizitem Token laden\n",
pipe = pipeline("text-generation", model="mistralai/Mistral-7B-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")


def generate_answer(query):
    relevant_chunks = get_relevant_chunks(query, top_k=3)
    context = "\n".join(relevant_chunks)
    prompt = f"Beantworte die Frage basierend auf diesem Kontext:\n \n{context}\n \n Frage: {query} \n Antwort:"

    response = pipe(prompt, max_new_tokens=128, do_sample=True, temperature=0.5)
    return response[0]["generated_text"]

In [ ]:
# Test\n",
frage = "Wie hoch ist der gesetzliche Mindestlohn?"
antwort = generate_answer(frage)
print("Antwort:", antwort)